In [14]:
from plotly import __version__
import plotly
import plotly.graph_objs as go
import pandas as pd

In [40]:
compressed_fifa_ratings = pd.read_csv('compressed_player_rating_dataset.csv', encoding='latin-1')
# compressed_fifa_ratings.set_index('player_url', inplace=True)

parsed_positions = compressed_fifa_ratings['position'].str.split(' ', expand=True)
parsed_positions.columns = ['main_position', 'position_2', 'position_3', 'position_4']
compressed_fifa_ratings = pd.concat([compressed_fifa_ratings, parsed_positions], axis=1)

compressed_fifa_ratings.drop(['Unnamed: 0', 'position'], axis=1, inplace=True)

just_overall_ratings = compressed_fifa_ratings[['player_url', 'age', 'main_position', 'nationality', 'team', 'overall']].drop_duplicates()
just_overall_ratings['age_next_year'] = just_overall_ratings['age'].apply(lambda x: x+1)
year_over_year_ratings = pd.merge(just_overall_ratings, just_overall_ratings, left_on=['player_url', 'age_next_year'],
                                  right_on=['player_url', 'age'], how='left').drop_duplicates()
year_over_year_ratings = year_over_year_ratings[['player_url', 'main_position_x', 'age_x', 'age_next_year_x', 'overall_x', 'overall_y', 'nationality_y', 'team_y']]
year_over_year_ratings.columns = ['player_url', 'main_position', 'age', 'age_next_year', 'overall', 'next_year_overall', 'nationality', 'team']
year_over_year_ratings.dropna(inplace=True)

year_over_year_ratings['diff'] = year_over_year_ratings['next_year_overall'] - year_over_year_ratings['overall']

year_over_year_ratings = year_over_year_ratings.loc[(year_over_year_ratings['age'] >= 16) & (year_over_year_ratings['age'] <= 38)]

In [41]:
# list out all available nationalities, teams and positions for the drop-down boxes
all_nationalities = sorted(list(set(year_over_year_ratings['nationality'])))
all_positions = sorted(list(set(year_over_year_ratings['main_position'])))
all_teams = sorted(list(set(year_over_year_ratings['team'])))

# return the items selected by the drop-down boxes
nationalities_chosen = ['Netherlands', 'Germany']
positions_chosen = ['CM']
teams_chosen = []
chosen_items = {'nationality': nationalities_chosen, 'main_position': positions_chosen, 'team': teams_chosen}

all_ages = sorted(list(set(year_over_year_ratings['age'])))

def filtered_aging_curves(year_over_year_ratings, chosen_items):
    # filter the overall dataset by the selected items, and delete any duplicate values
    for key, value in chosen_items.items():
        if len(value) > 0:
            year_over_year_ratings = year_over_year_ratings.loc[year_over_year_ratings[key].isin(value)]
    year_over_year_ratings.drop_duplicates(subset=['player_url', 'age', 'age_next_year'], inplace=True)

    # group by the relevant columns
    avg_rating_changes = year_over_year_ratings.groupby(['age', 'age_next_year'], as_index=False)['diff'].mean()
    avg_rating_changes.columns = ['age', 'age_next_year', 'diff']

    avg_rating_changes['cum_sum'] = avg_rating_changes['diff'].cumsum()
#     maximum_rating = avg_rating_changes['cum_sum'].max()
#     avg_rating_changes['aging_curve_rating'] = avg_rating_changes['cum_sum'] - maximum_rating
    
    aging_curve_data = avg_rating_changes[['age', 'age_next_year', 'cum_sum']]
    
    return list(aging_curve_data['age']), list(aging_curve_data['cum_sum'])

In [43]:
plotly.offline.init_notebook_mode(connected=True)

layout = go.Layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230, 230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict(x=1, y=1, z=0.7),
        aspectmode = 'manual'
    )
)


nationality_buttons = []

for i in all_nationalities:
    nationality_buttons.append(
        dict(args=[list(filtered_aging_curves(year_over_year_ratings, i)['age']), 
                   list(filtered_aging_curves(year_over_year_ratings, i)['cum_sum'])],
             label=i, method='restyle')
    )

layout = go.Layout(

    annotations=[dict(text='Change Nationality',
                      x=-0.25, y=0.83,
                      xref='paper', yref='paper',
                      showarrow=False)],
    updatemenus=list([
        dict(x=-0.1, y=0.7,
             yanchor='middle',
             bgcolor='#ff0000',
             buttons=list(nationality_buttons)),
    ])
)
        

plotly.offline.iplot({
    "data": [go.Scatter(x=avg_rating_changes['age'], y=avg_rating_changes['aging_curve_rating'])],
    "layout": layout
})

AttributeError: 'str' object has no attribute 'items'

In [81]:
import pandas as pd
import plotly.plotly as py

from ipywidgets import widgets
from IPython.display import display
import plotly.graph_objs as go
from plotly.widgets import GraphWidget

plotly.offline.init_notebook_mode(connected=True)


nationalities = widgets.Dropdown(
    description='Nationality:   ',
    value='Netherlands',
    options=all_nationalities,
)

positions = widgets.Dropdown(
    description='Position:',
    value='ST',
    options=all_positions,
)

def filtered_aging_curves(year_over_year_ratings, chosen_items):
    # filter the overall dataset by the selected items, and delete any duplicate values
    for key, value in chosen_items.items():
        if len(value) > 0:
            year_over_year_ratings = year_over_year_ratings.loc[year_over_year_ratings[key].isin(value)]
    year_over_year_ratings.drop_duplicates(subset=['player_url', 'age', 'age_next_year'], inplace=True)

    # group by the relevant columns
    avg_rating_changes = year_over_year_ratings.groupby(['age', 'age_next_year'], as_index=False)['diff'].mean()
    avg_rating_changes.columns = ['age', 'age_next_year', 'diff']

    avg_rating_changes['cum_sum'] = avg_rating_changes['diff'].cumsum()
#     maximum_rating = avg_rating_changes['cum_sum'].max()
#     avg_rating_changes['aging_curve_rating'] = avg_rating_changes['cum_sum'] - maximum_rating
    
    aging_curve_data = avg_rating_changes[['age', 'age_next_year', 'cum_sum']]
    
    return list(aging_curve_data['age']), list(aging_curve_data['cum_sum'])


default_age_series, default_aging_series = filtered_aging_curves(year_over_year_ratings, {'nationality': [], 'main_position': [], 'team': []})
scatter_plot = go.Scatter(x=default_age_series, y=default_aging_series)

layout = go.Layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230, 230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict(x=1, y=1, z=0.7),
        aspectmode = 'manual'
    )
)

g = go.FigureWidget(data=[scatter_plot], layout=layout)

def validate():
    if nationalities.value in all_nationalities and positions.value in all_positions:
        return True
    else:
        return False


def response(change):
    if validate():
        changes = {'nationality': [nationalities.value], 'main_position': [positions.value]}
        x, y = filtered_aging_curves(year_over_year_ratings, changes)
        with g.batch_update():
            g.data[0].x = x
            g.data[0].y = y


nationalities.observe(response, names="value")
positions.observe(response, names="value")

filters = widgets.VBox([nationalities, positions])
visualization = widgets.HBox([filters, g])
display(filters)
display(g)

A Jupyter Widget

A Jupyter Widget

before (16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38)
(16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38)
before (27, 28)
(27, 28)
